In [ ]:
import pyodbc
from pyproj import CRS, Transformer
from shapely.wkt import loads, dumps

# --- Helper function for transformation ---
from shapely.ops import transform as shapely_transform

def transform_geom(geom, transformer):
    return shapely_transform(transformer.transform, geom)

# 1. Define CRSs and the transformer
crs_3857 = CRS("EPSG:3857")
crs_4326 = CRS("EPSG:4326")
transformer = Transformer.from_crs(crs_3857, crs_4326, always_xy=True)

# --- Database Connection ---
DB_USERNAME="[username]"
DB_PASSWORD="[password]"
DB_SERVER="[servername]"
DB_DATABASE="LVMPD_Crime_FF"

conn = pyodbc.connect(
    driver='{ODBC Driver 18 for SQL Server}'
    , server = DB_SERVER
    , database = DB_DATABASE
    , uid = DB_USERNAME
    , pwd = DB_PASSWORD
    , TrustServerCertificate = 'yes'
)
cursor = conn.cursor()
cursor.execute("SELECT etl_file_xid, Geom.STAsText() FROM LVMPD_Crime_FF.dbo.LVMPD_Area_Commands")
rows = cursor.fetchall()

# 3. Transform
for row in rows:
    primary_key = row[0]
    wkt_3857 = row[1]

    try:
        # Load the WKT from the database into a shapely object
        geom_3857 = loads(wkt_3857)

        # Transform all points in the geometry
        # This uses a helper function to handle different geometry types
        geom_4326 = transform_geom(geom_3857, transformer)

        # Convert back to WKT for saving
        wkt_4326 = dumps(geom_4326)

        # 4. Load the data back to your data warehouse
        update_sql = "UPDATE LVMPD_Crime_FF.dbo.LVMPD_Area_Commands SET Geom = geometry::STGeomFromText(?, 4326) WHERE etl_file_xid = ?"
        cursor.execute(update_sql, wkt_4326, primary_key)

    except Exception as e:
        print(f"Failed to transform row {primary_key}: {e}")

# Don't forget to commit and close connections
conn.commit()
cursor.close()
conn.close()

